In [3]:
import pandas as pd
import numpy as np
from helpers.data_load import load_data_from_txt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import warnings
warnings.filterwarnings('ignore')



c:\Users\79991\AppData\Local\Programs\Python\Python37\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [4]:
import time
import datetime

In [5]:
data = load_data_from_txt(path='data/data.txt')

In [6]:
data

,EURUSD,numdate,time,open,max,min,close,vol
EURUSD,5,20200701,500,1.12332,1.12364,1.12310,1.12330,245
EURUSD,5,20200701,1000,1.12327,1.12356,1.12310,1.12330,1414
EURUSD,5,20200701,1500,1.12331,1.12351,1.12310,1.12322,1317
EURUSD,5,20200701,2000,1.12322,1.12340,1.12300,1.12323,992
EURUSD,5,20200701,2500,1.12321,1.12346,1.12297,1.12314,982
...,...,...,...,...,...,...,...,...
EURUSD,5,20200930,234500,1.17256,1.17275,1.17230,1.17249,2480
EURUSD,5,20200930,235000,1.17241,1.17258,1.17220,1.17245,2192
EURUSD,5,20200930,235500,1.17237,1.17248,1.17180,1.17207,3880
EURUSD,5,20201001,0,1.17216,1.17235,1.17185,1.17190,4947


In [10]:
test_df = data.sample(1000)

In [7]:
def divide_numdate(num_date):
    return f'{num_date[0:4]}:{num_date[4:6]}:{num_date[6:8]}'

def divide_timedate(time_date, pairs_count=3):
    time_date = (6 - len(time_date))*'0' + time_date
    time_date = ':'.join([time_date[2*i:2*(i+1)] for i in range(pairs_count)])
    return time_date

def to_datetime_index_date(index_date):
    return datetime.datetime.strptime(index_date, '%Y:%m:%d %H:%M:%S')
    

In [37]:
def prep_dataframe(dataframe):
    non_inplace_df = dataframe.copy(deep=True)
     
    non_inplace_df.numdate = non_inplace_df.numdate.astype(str)
    non_inplace_df.time = non_inplace_df.time.astype(str)
    
    non_inplace_df['hms'] = non_inplace_df.time.apply(divide_timedate)
    non_inplace_df['ymd'] = non_inplace_df.numdate.apply(divide_numdate)
    
    non_inplace_df['index_date'] = non_inplace_df.ymd + ' ' + non_inplace_df.hms
    non_inplace_df.index_date.apply(to_datetime_index_date)
    
    non_inplace_df.drop(['ymd', 'hms', 'EURUSD', 'numdate', 'time'], axis=1, inplace=True)
    
    non_inplace_df.set_index('index_date', inplace=True)

    
    return non_inplace_df